## 16S preliminary metaanalysis of sociodemographic associations with childhood gut microbiomes

This is a metaanalysis of publicly available 16S rRNA sequencing datasets from the gut microbiomes of pre-adolescent children. The goal is to see if sociodemographic factors are related to gut microbiome composition.

This analysis was run in QIIME2-2021.4.

### Importing data

Each study is individually imported, as different sequencing protocols and primer sets were used in each study. Code for importing each study is below:

In [ ]:
#Importing sequences from Herman et al., 2019
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/herman

qiime tools import --type EMPPairedEndSequences --input-path ${project} --output-path ${project}/herman_raw_seqs.qza

qiime demux emp-paired --m-barcodes-file ${project}/herman_metadata.txt --m-barcodes-column BarcodeSequence \
    --p-rev-comp-mapping-barcodes --i-seqs ${project}/herman_raw_seqs.qza \
    --o-per-sample-sequences ${project}/herman-paired-end-demux.qza \
    --o-error-correction-details ${project}/herman-demux-details.qza
    
qiime demux summarize --i-data ${project}/herman-paired-end-demux.qza \
    --o-visualization ${project}/herman-paired-end-demux.qzv

In [ ]:
#Importing sequences from Chu et al., 2016
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/chu

cd ${project}

qiime tools import --type 'SampleData[SequencesWithQuality]' --input-path ${project}/Manifest.txt \
    --output-path chu-single-end-demux.qza --input-format SingleEndFastqManifestPhred33V2
    
cd

qiime demux summarize --i-data ${project}/chu-single-end-demux.qza \
    --o-visualization ${project}/chu-single-end-demux.qzv

In [ ]:
#Importing sequences from Planer et al., 2016
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/planer

cd ${project}

qiime tools import --type 'SampleData[PairedEndSequencesWithQuality]' --input-path ${project}/Manifest.tsv \
    --output-path planer-paired-end-demux.qza --input-format PairedEndFastqManifestPhred33V2
    
cd

qiime demux summarize --i-data ${project}/planer-paired-end-demux.qza \
    --o-visualization ${project}/planer-paired-end-demux.qzv

In [ ]:
#Importing sequences from Kim type 1 diabetes study (QIITA 11129)
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/kim_t1d

cd ${project}

qiime tools import --type 'EMPPairedEndSequences' \
    --input-path ${project}/study_raw_data_11129_070221-125259/FASTQ/31359 \
    --output-path ${project}/kim-t1d-paired-end.qza
    
cd

qiime tools peek ${project}/kim-t1d-paired-end.qza

qiime demux emp-paired --i-seqs ${project}/kim-t1d-paired-end.qza \
    --m-barcodes-file ${project}/study_raw_data_11129_070221-125259/mapping_files/31359_mapping_file.txt \
    --m-barcodes-column barcode \
    --p-rev-comp-barcodes --p-rev-comp-mapping-barcodes \
    --o-per-sample-sequences ${project}/kim-t1d-paired-end-demux.qza \
    --o-error-correction-details ${project}/demux-details.qza

qiime demux summarize --i-data ${project}/kim-t1d-paired-end-demux.qza \
    --o-visualization ${project}/kim-t1d-paired-end-demux.qzv


In [ ]:
#Importing sequences from MGD time series study (QIITA 10894)
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/MGD_infant_timeseries

cd ${project}

qiime tools import --type 'SampleData[SequencesWithQuality]' --input-path ${project}/Manifest.tsv \
    --output-path mgd-single-end-demux.qza --input-format SingleEndFastqManifestPhred33V2
    
cd

qiime demux summarize --i-data ${project}/mgd-single-end-demux.qza \
    --o-visualization ${project}/mgd-single-end-demux.qzv

In [ ]:
#Importing sequences from Levin et al., 2016
#Using forward reads only, because some sequences have length 0 in multiple files
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/levin

cd ${project}

qiime tools import --type 'SampleData[SequencesWithQuality]' --input-path ${project}/Manifest.tsv \
    --output-path levin-single-end-demux.qza --input-format SingleEndFastqManifestPhred33V2
    
cd

qiime demux summarize --i-data ${project}/levin-single-end-demux.qza \
    --o-visualization ${project}/levin-single-end-demux.qzv

In [ ]:
#Importing sequences from Robinson et al., 2017
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/robinson

cd ${project}

qiime tools import --type 'SampleData[PairedEndSequencesWithQuality]' --input-path ${project}/Manifest.tsv \
    --output-path robinson-paired-end-demux.qza --input-format PairedEndFastqManifestPhred33V2
    
cd

qiime demux summarize --i-data ${project}/robinson-paired-end-demux.qza \
    --o-visualization ${project}/robinson-paired-end-demux.qzv

### Running DADA2

Each study was individually run through DADA2, per the QIIME developers' suggestions. Trimming and truncation parameters were set individually for each study due to differences in primer sets and sequencing quality.

In [ ]:
#DADA2 for Herman et al., 2019 sequences
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/herman

qiime dada2 denoise-paired --i-demultiplexed-seqs ${project}/herman-paired-end-demux.qza \
    --p-trunc-len-f 150 --p-trunc-len-r 150 --p-trim-left-f 19 --p-trim-left-r 20 --p-n-threads 2 \
    --o-table ${project}/herman-table.qza \
    --o-representative-sequences ${project}/herman-rep-seqs.qza \
    --o-denoising-stats ${project}/herman-dada2-stats.qza

qiime metadata tabulate --m-input-file ${project}/herman-dada2-stats.qza \
    --o-visualization ${project}/herman-dada2-stats.qzv

qiime feature-table summarize --i-table ${project}/herman-table.qza \
    --o-visualization ${project}/herman-table.qzv \
    --m-sample-metadata-file ${project}/herman_metadata.txt

In [ ]:
#DADA2 for Chu et al., 2016 sequences
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/chu

qiime dada2 denoise-pyro --i-demultiplexed-seqs ${project}/chu-single-end-demux.qza \
    --p-trunc-len 450 --p-trim-left 17 --p-n-threads 2 \
    --o-table ${project}/chu-table.qza \
    --o-representative-sequences ${project}/chu-rep-seqs.qza \
    --o-denoising-stats ${project}/chu-dada2-stats.qza

qiime metadata tabulate --m-input-file ${project}/chu-dada2-stats.qza \
    --o-visualization ${project}/chu-dada2-stats.qzv

qiime feature-table summarize --i-table ${project}/chu-table.qza \
    --o-visualization ${project}/chu-table.qzv \
    --m-sample-metadata-file ${project}/chu_metadata.txt

In [ ]:
#DADA2 for Planer et al., 2016 sequences
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/planer

qiime dada2 denoise-paired --i-demultiplexed-seqs ${project}/planer-paired-end-demux.qza \
    --p-trunc-len-f 220 --p-trunc-len-r 220 --p-trim-left-f 19 --p-trim-left-r 20 --p-n-threads 2 \
    --o-table ${project}/planer-table.qza \
    --o-representative-sequences ${project}/planer-rep-seqs.qza \
    --o-denoising-stats ${project}/planer-dada2-stats.qza

qiime metadata tabulate --m-input-file ${project}/planer-dada2-stats.qza \
    --o-visualization ${project}/planer-dada2-stats.qzv

qiime feature-table summarize --i-table ${project}/planer-table.qza \
    --o-visualization ${project}/planer-table.qzv \
    --m-sample-metadata-file ${project}/planer_metadata.txt

In [ ]:
#DADA2 for Kim type 1 diabetes study (QIITA 11129)
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/kim_t1d

qiime dada2 denoise-paired --i-demultiplexed-seqs ${project}/kim-t1d-paired-end-demux.qza \
    --p-trunc-len-f 150 --p-trunc-len-r 150 --p-trim-left-f 0 --p-trim-left-r 0 --p-n-threads 2 \
    --o-table ${project}/kim-t1d-table.qza \
    --o-representative-sequences ${project}/kim-t1d-rep-seqs.qza \c
    --o-denoising-stats ${project}/kim-t1d-dada2-stats.qza

qiime metadata tabulate --m-input-file ${project}/kim-t1d-dada2-stats.qza \
    --o-visualization ${project}/kim-t1d-dada2-stats.qzv

qiime feature-table summarize --i-table ${project}/kim-t1d-table.qza \
    --o-visualization ${project}/kim-t1d-table.qzv \
    --m-sample-metadata-file ${project}/kim_metadata.txt

In [ ]:
#DADA2 for MGD time series study (QIITA 10894)
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/MGD_infant_timeseries

qiime dada2 denoise-single --i-demultiplexed-seqs ${project}/mgd-single-end-demux.qza \
    --p-trunc-len 125 --p-trim-left 19 --p-n-threads 2 \
    --o-table ${project}/mgd-table.qza \
    --o-representative-sequences ${project}/mgd-rep-seqs.qza \
    --o-denoising-stats ${project}/mgd-dada2-stats.qza

qiime metadata tabulate --m-input-file ${project}/mgd-dada2-stats.qza \
    --o-visualization ${project}/mgd-dada2-stats.qzv

qiime feature-table summarize --i-table ${project}/mgd-table.qza \
    --o-visualization ${project}/mgd-table.qzv \
    --m-sample-metadata-file ${project}/mgd_timeseries_metadata.txt

In [ ]:
#DADA2 for Levin et al., 2016 sequences
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/levin

qiime dada2 denoise-single --i-demultiplexed-seqs ${project}/levin-single-end-demux.qza \
    --p-trunc-len 250 --p-trim-left 19 --p-n-threads 2 \
    --o-table ${project}/levin-table.qza \
    --o-representative-sequences ${project}/levin-rep-seqs.qza \
    --o-denoising-stats ${project}/levin-dada2-stats.qza

qiime metadata tabulate --m-input-file ${project}/levin-dada2-stats.qza \
    --o-visualization ${project}/levin-dada2-stats.qzv

qiime feature-table summarize --i-table ${project}/levin-table.qza \
    --o-visualization ${project}/levin-table.qzv \
    --m-sample-metadata-file ${project}/levin_metadata.txt

In [ ]:
#DADA2 for Robinson et al., 2017 sequences
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/robinson

qiime dada2 denoise-paired --i-demultiplexed-seqs ${project}/robinson-paired-end-demux.qza \
    --p-trunc-len-f 248 --p-trunc-len-r 200 --p-trim-left-f 19 --p-trim-left-r 20 --p-n-threads 2 \
    --o-table ${project}/robinson-table.qza \
    --o-representative-sequences ${project}/robinson-rep-seqs.qza \
    --o-denoising-stats ${project}/robinson-dada2-stats.qza

qiime metadata tabulate --m-input-file ${project}/robinson-dada2-stats.qza \
    --o-visualization ${project}/robinson-dada2-stats.qzv

qiime feature-table summarize --i-table ${project}/robinson-table.qza \
    --o-visualization ${project}/robinson-table.qzv \
    --m-sample-metadata-file ${project}/robinson_metadata.txt

### Assigning taxonomy

Taxonomy was assigned using a Naive-Bayesian classifier trained on the Greengenes 13_8 99% OTU full-length 16S sequence database. Taxonomy was assigned for each study individually, and then for merged datasets (see below).

Pre-trained classifiers were obtained from the QIIME2 website:
Bokulich, N.A., Robeson, M., Dillon, M.R. bokulich-lab/RESCRIPt. Zenodo. http://doi.org/10.5281/zenodo.3891931
Bokulich, N.A., Kaehler, B.D., Rideout, J.R. et al. Optimizing taxonomic classification of marker-gene amplicon sequences with QIIME 2’s q2-feature-classifier plugin. Microbiome 6, 90 (2018). https://doi.org/10.1186/s40168-018-0470-z

In [ ]:
#Taxonomy for Herman et al., 2019 sequences
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/herman

qiime feature-classifier classify-sklearn --i-classifier gg-13-8-99-nb-classifier.qza \
    --i-reads ${project}/herman-rep-seqs.qza --o-classification ${project}/herman-taxonomy.qza

In [ ]:
#Taxonomy for Chu et al., 2016 sequences
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/chu

qiime feature-classifier classify-sklearn --i-classifier gg-13-8-99-nb-classifier.qza \
    --i-reads ${project}/chu-rep-seqs.qza --o-classification ${project}/chu-taxonomy.qza

In [ ]:
#Taxonomy for Planer et al., 2016 sequences
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/planer

qiime feature-classifier classify-sklearn --i-classifier gg-13-8-99-nb-classifier.qza \
    --i-reads ${project}/planer-rep-seqs.qza --o-classification ${project}/planer-taxonomy.qza

In [ ]:
#Taxonomy for Kim type 1 diabetes study (QIITA 11129)
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/kim_t1d

qiime feature-classifier classify-sklearn --i-classifier gg-13-8-99-nb-classifier.qza \
    --i-reads ${project}/kim-t1d-rep-seqs.qza --o-classification ${project}/kim-t1d-taxonomy.qza

In [ ]:
#Taxonomy for MGD time series study (QIITA 10894)
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/MGD_infant_timeseries

qiime feature-classifier classify-sklearn --i-classifier gg-13-8-99-nb-classifier.qza \
    --i-reads ${project}/mgd-rep-seqs.qza --o-classification ${project}/mgd-taxonomy.qza

In [ ]:
#Taxonomy for Levin et al., 2016
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/levin

qiime feature-classifier classify-sklearn --i-classifier gg-13-8-99-nb-classifier.qza \
    --i-reads ${project}/levin-rep-seqs.qza --o-classification ${project}/levin-taxonomy.qza

In [ ]:
#Taxonomy for Robinson et al., 2017
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children/robinson

qiime feature-classifier classify-sklearn --i-classifier gg-13-8-99-nb-classifier.qza \
    --i-reads ${project}/robinson-rep-seqs.qza --o-classification ${project}/robinson-taxonomy.qza

### Merging datasets

Output feature table and representative sequences from all studies were merged. The q2-fragment-insertion QIIME2 plugin was used to create phylogenetic trees with the merged datasets. The merged table was filtered to remove sequences not present in the insertion tree.

In [ ]:
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children

qiime feature-table merge --i-tables ${project}/herman/herman-table.qza --i-tables ${project}/chu/chu-table.qza \
     --i-tables ${project}/planer/planer-table.qza --i-tables ${project}/kim_t1d/kim-t1d-table.qza \
     --i-tables ${project}/MGD_infant_timeseries/mgd-table.qza ${project}/levin/levin-table.qza \
     --i-tables ${project}/robinson/robinson-table.qza --o-merged-table ${project}/merged-table.qza

qiime feature-table merge-seqs --i-data ${project}/herman/herman-rep-seqs.qza \
    --i-data ${project}/chu/chu-rep-seqs.qza --i-data ${project}/planer/planer-rep-seqs.qza \
    --i-data ${project}/kim_t1d/kim-t1d-rep-seqs.qza --i-data ${project}/MGD_infant_timeseries/mgd-rep-seqs.qza \
    --i-data ${project}/levin/levin-rep-seqs.qza --i-data ${project}/robinson/robinson-rep-seqs.qza \
    --o-merged-data ${project}/merged-rep-seqs.qza 

wget -O "sepp-refs-gg-13-8.qza" \
  "https://data.qiime2.org/2020.11/common/sepp-refs-gg-13-8.qza"
  
qiime fragment-insertion sepp --i-representative-sequences ${project}/merged-rep-seqs.qza \
    --i-reference-database /Users/elizabethmallott/sepp-refs-gg-13-8.qza --o-tree ${project}/insertion-tree.qza \
    --o-placements ${project}/insertion-placements.qza --p-threads 2

qiime fragment-insertion filter-features --i-table ${project}/merged-table.qza \
    --i-tree ${project}/insertion-tree.qza --o-filtered-table ${project}/insertion-filtered-merged-table.qza \
    --o-removed-table ${project}/removed-insertion-merged-table.qza

### Taxonomic assigned of merged datasets

We assigned taxonomy to the merged dataset two ways. First, using a Naive-Bayesian classifier trained on the Greengenes 13_8 99% OTU full-length 16S sequence database, as above. Second, we used the experimental OTU classification method based on the results of the fragment insertion method results. Mitochondrial and chloroplast sequences were removed based on the results of both methods.

In [ ]:
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children

qiime feature-classifier classify-sklearn --i-classifier gg-13-8-99-nb-classifier.qza \
    --i-reads ${project}/merged-rep-seqs.qza --o-classification ${project}/merged-taxonomy.qza
    
qiime fragment-insertion classify-otus-experimental --i-representative-sequences ${project}/merged-rep-seqs.qza \
    --i-tree ${project}/insertion-tree.qza --i-reference-taxonomy ${project}/taxonomy_gg99.qza \
    --o-classification ${project}/merged-otu-taxonomy.qza

### Filtering table

The merged table was filtered to remove mitochondrial and chloroplast sequences, as well as remove control sample or non-fecal samples from some datasets. The table was then filtered to remove samples lacking specific metadata categories prior to downstream analysis.

In [ ]:
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children

qiime taxa filter-table --i-table ${project}/insertion-filtered-merged-table.qza \
    --i-taxonomy ${project}/merged-taxonomy.qza --p-exclude mitochondria,chloroplast \
    --o-filtered-table ${project}/merged-table-nomito-nochloro.qza

qiime taxa filter-table --i-table ${project}/insertion-filtered-merged-table.qza \
    --i-taxonomy ${project}/merged-otu-taxonomy.qza --p-exclude mitochondria,chloroplast \
    --o-filtered-table ${project}/merged-table-nomito-nochloro-otu.qza
    
qiime feature-table filter-samples --i-table ${project}/merged-table-nomito-nochloro.qza \
    --m-metadata-file ${project}/metadata-combined.txt --o-filtered-table ${project}/merged-table-filtered.qza
    
qiime feature-table filter-samples --i-table ${project}/merged-table-nomito-nochloro-otu.qza \
    --m-metadata-file ${project}/metadata-combined.txt --o-filtered-table ${project}/merged-table-filtered-otu.qza
    
qiime feature-table summarize --i-table ${project}/merged-table-filtered.qza \
    --o-visualization ${project}/merged-table-filtered.qzv \
    --m-sample-metadata-file ${project}/metadata-combined.txt

qiime feature-table summarize --i-table ${project}/merged-table-filtered-otu.qza \
    --o-visualization ${project}/merged-table-filtered-otu.qzv \
    --m-sample-metadata-file ${project}/metadata-combined.txt

qiime feature-table filter-samples --i-table ${project}/merged-table-nomito-nochloro.qza \
    --m-metadata-file ${project}/metadata-combined-re.txt \
    --o-filtered-table ${project}/merged-table-filtered-re.qza

qiime feature-table filter-samples --i-table ${project}/merged-table-nomito-nochloro.qza \
    --m-metadata-file ${project}/metadata-combined-re-age.txt \
    --o-filtered-table ${project}/merged-table-filtered-re-age.qza

qiime feature-table filter-samples --i-table ${project}/merged-table-nomito-nochloro.qza \
    --m-metadata-file ${project}/metadata-combined-re-age-sex.txt \
    --o-filtered-table ${project}/merged-table-filtered-re-age-sex.qza

qiime feature-table filter-samples --i-table ${project}/merged-table-nomito-nochloro.qza \
    --m-metadata-file ${project}/metadata-combined-re-age-sex-delivery.txt \
    --o-filtered-table ${project}/merged-table-filtered-re-age-sex-delivery.qza

qiime feature-table filter-samples --i-table ${project}/merged-table-nomito-nochloro.qza \
    --m-metadata-file ${project}/metadata-combined-re-age-sex-feeding.txt \
    --o-filtered-table ${project}/merged-table-filtered-re-age-sex-feeding.qza

qiime feature-table filter-samples --i-table ${project}/merged-table-nomito-nochloro.qza \
    --m-metadata-file ${project}/metadata-combined-re-age-sex-delivery-feeding.txt \
    --o-filtered-table ${project}/merged-table-filtered-re-age-sex-delivery-feeding.qza

### Alpha and beta diversity analyses

Alpha and beta diversity metrics were calculated in QIIME2, and then results were output to R for statistical analysis.

In [ ]:
project=/Users/elizabethmallott/Dropbox/Projects/VMI/children

qiime diversity core-metrics-phylogenetic --i-phylogeny ${project}/insertion-tree.qza \
    --i-table ${project}/merged-table-filtered.qza --p-sampling-depth 5000 \
    --m-metadata-file ${project}/metadata-combined.txt \
    --output-dir ${project}/core-metrics-5000

qiime diversity core-metrics-phylogenetic --i-phylogeny ${project}/insertion-tree.qza \
    --i-table ${project}/merged-table-filtered-re.qza --p-sampling-depth 5000 \
    --m-metadata-file ${project}/metadata-combined-re.txt \
    --output-dir ${project}/core-metrics-5000-re

qiime diversity core-metrics-phylogenetic --i-phylogeny ${project}/insertion-tree.qza \
    --i-table ${project}/merged-table-filtered-re-age.qza --p-sampling-depth 5000 \
    --m-metadata-file ${project}/metadata-combined-re-age.txt \
    --output-dir ${project}/core-metrics-5000-re-age

qiime diversity core-metrics-phylogenetic --i-phylogeny ${project}/insertion-tree.qza \
    --i-table ${project}/merged-table-filtered-re-age-sex.qza --p-sampling-depth 5000 \
    --m-metadata-file ${project}/metadata-combined-re-age-sex.txt \
    --output-dir ${project}/core-metrics-5000-re-age-sex

qiime diversity core-metrics-phylogenetic --i-phylogeny ${project}/insertion-tree.qza \
    --i-table ${project}/merged-table-filtered-re-age-sex-delivery.qza --p-sampling-depth 5000 \
    --m-metadata-file ${project}/metadata-combined-re-age-sex-delivery.txt \
    --output-dir ${project}/core-metrics-5000-re-age-sex-delivery

qiime diversity core-metrics-phylogenetic --i-phylogeny ${project}/insertion-tree.qza \
    --i-table ${project}/merged-table-filtered-re-age-sex-feeding.qza --p-sampling-depth 5000 \
    --m-metadata-file ${project}/metadata-combined-re-age-sex-feeding.txt \
    --output-dir ${project}/core-metrics-5000-re-age-sex-feeding

qiime diversity core-metrics-phylogenetic --i-phylogeny ${project}/insertion-tree.qza \
    --i-table ${project}/merged-table-filtered-re-age-sex-delivery-feeding.qza --p-sampling-depth 5000 \
    --m-metadata-file ${project}/metadata-combined-re-age-sex-delivery-feeding.txt \
    --output-dir ${project}/core-metrics-5000-re-age-sex-delivery-feeding

qiime tools extract --input-path ${project}/core-metrics-5000/unweighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000

qiime tools extract --input-path ${project}/core-metrics-5000-/weighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000

qiime tools extract --input-path ${project}/core-metrics-5000-re/unweighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re

qiime tools extract --input-path ${project}/core-metrics-5000-re/weighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re

qiime tools extract --input-path ${project}/core-metrics-5000-re-age/unweighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re-age

qiime tools extract --input-path ${project}/core-metrics-5000-re-age/weighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re-age

qiime tools extract --input-path ${project}/core-metrics-5000-re-age-sex/unweighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re-age-sex

qiime tools extract --input-path ${project}/core-metrics-5000-re-age-sex/weighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re-age-sex

qiime tools extract --input-path ${project}/core-metrics-5000-re-age-sex-delivery/unweighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re-age-sex-delivery

qiime tools extract --input-path ${project}/core-metrics-5000-re-age-sex-delivery/weighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re-age-sex-delivery

qiime tools extract --input-path ${project}/core-metrics-5000-re-age-sex-feeding/unweighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re-age-sex-feeding

qiime tools extract --input-path ${project}/core-metrics-5000-re-age-sex-feeding/weighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re-age-sex-feeding

qiime tools extract --input-path ${project}/core-metrics-5000-re-age-sex-delivery-feeding/unweighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re-age-sex-delivery-feeding

qiime tools extract --input-path ${project}/core-metrics-5000-re-age-sex-delivery-feeding/weighted_unifrac_distance_matrix.qza \
    --output-path ${project}/core-metrics-5000-re-age-sex-delivery-feeding

qiime tools export --input-path ${project}/core-metrics-5000/faith_pd_vector.qza \
    --output-path ${project}/alpha_out

qiime tools export --input-path ${project}/core-metrics-5000/shannon_vector.qza \
    --output-path ${project}/alpha_out

qiime tools export --input-path ${project}/core-metrics-5000/observed_features_vector.qza \
    --output-path ${project}/alpha_out

### Taxa bar plots

Taxa bar plots were constructed for both the taxonomic assignment methods. Then feature tables were converted to relative abundance and exported.

In [ ]:
qiime taxa barplot --i-table ${project}/merged-table-filtered.qza --i-taxonomy ${project}/merged-taxonomy.qza \
    --m-metadata-file ${project}/metadata-combined.txt --o-visualization ${project}/taxa_plots_merged_filtered.qzv

qiime taxa barplot --i-table ${project}/merged-table-filtered-otu.qza --i-taxonomy ${project}/merged-otu-taxonomy.qza \
    --m-metadata-file ${project}/metadata-combined.txt --o-visualization ${project}/taxa_plots_merged_filtered-otu.qzv
    
qiime feature-table relative-frequency --i-table ${project}/merged-table-filtered.qza \
    --o-relative-frequency-table ${project}/merged-table-filtered-relab.qza

qiime tools export --input-path ${project}/merged-table-filtered-relab.qza --output-path ${project}/exported

qiime tools export --input-path ${project}/merged-taxonomy.qza --output-path ${project}/exported

cp ${project}/exported/taxonomy.tsv ${project}/biom-taxonomy.tsv #edit header to #OTUID taxonomy confidence

biom add-metadata -i ${project}/exported/feature-table.biom -o ${project}/table-with-taxonomy.biom \
    --observation-metadata-fp ${project}/biom-taxonomy.tsv --sc-separated taxonomy

biom convert -i ${project}/table-with-taxonomy.biom -o ${project}/feature-table-full.tsv \
    --to-tsv --header-key taxonomy